<img src="L1/0.png" alt="Drawing" style="width: 400px;"/>

# Localization (Monte Carlo localization method)

Contents:
* Sense
* Motion
* Sense and Motion

<img src="L1/1.png" alt="Drawing" style="width: 400px;"/>

## Sense or measurement

<img src="L1/2.png" alt="Drawing" style="width: 400px;"/>

* State 1: Maximum confusion with uniform probabilty.
* State 2: Increase confidence with measurement $Z$. Normalize $p(X_i|Z)$.  
Note: $p(X_i|Z)$ == $p(H|D)$, H: Hypothesis; D: Data.

In [18]:
#######################
# Multiple measurements
#######################
p = [0.2, 0.2, 0.2, 0.2, 0.2]
world = ['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
#measurements = ['red']
pHit = 0.6
pMiss = 0.2

def sense(p, Z):
    # update the confidence
    q=[]
    for i in range(len(p)):
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1-hit) * pMiss)) 
    
    # normalize
    s = sum(q)
    for i in range(len(p)):
        q[i] = q[i]/s
    return q


# loop over series of measurements
for k in range(len(measurements)):
    p = sense(p, measurements[k])
    print p

#run this twice and get back the uniform distribution

[0.1111111111111111, 0.3333333333333332, 0.3333333333333332, 0.1111111111111111, 0.1111111111111111]
[0.20000000000000004, 0.19999999999999996, 0.19999999999999996, 0.20000000000000004, 0.20000000000000004]


## Motion

### Exact motion

<img src="L1/3.png" alt="Drawing" style="width: 300px;"/>

In [19]:
def move_exact(p, U):
    q= [ ]
    for i in range(len(p)):
        q.append(p[(i-U) % len (p)])
    return q

for k in range(len(measurements)):
    p = sense(p, measurements [k])
    print move_exact(p, 1)

[0.11111111111111113, 0.11111111111111113, 0.33333333333333326, 0.33333333333333326, 0.11111111111111113]
[0.2, 0.2, 0.19999999999999993, 0.19999999999999993, 0.2]


### Inexact motion

<img src="L1/4.png" alt="Drawing" style="width: 300px;"/>

Example:
<img src="L1/5.png" alt="Drawing" style="width: 300px;"/>
* Note: Theorem of Total Probability

In [20]:
p = [0, 1, 0, 0, 0]
world = ['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2

#Add exact probability
pExact = 0.8
#Add overshoot probability
pOvershoot = 0.1
pUndershoot = 0.1

def move_inexact(p, U):
    q= []
    for i in range(len(p)):
        s = pExact * p[(i-U) % len(p)]
        s = s + pOvershoot * p[(i-U-1) % len(p)]
        s = s + pUndershoot * p[(i-U+1) % len(p)]
        q.append(s)
    return q

for k in range(len(measurements)):
    p = sense(p, measurements [k])
    print move_inexact(p, 1)

[0.0, 0.1, 0.8, 0.1, 0.0]
[0.0, 0.1, 0.8, 0.1, 0.0]


* Limit distribution

<img src="L1/6.png" alt="Drawing" style="width: 300px;"/>

In [21]:
p = [0, 0.5, 0.3, 0.2, 0]

for k in range(1000):
    p = move_inexact(p, 1)
print p

[0.2000000000000035, 0.2000000000000035, 0.2000000000000035, 0.2000000000000035, 0.2000000000000035]


## Robot Sense and Movement

Every time it moves it **loses** information and every time it senses it **gains** information. 

**Entropy** is a measure of the information that a distribution has.

<img src="L1/7.png" alt="Drawing" style="width: 400px;"/>

Our Monte Carlo localization procedure can be written as a series of steps:
1. Start with a belief set about our current location.
2. Multiply this distribution by the results of our sense measurement.
3. Normalize the resulting distribution.
4. Move by performing a convolution. This sounds more complicated than it really is: this
step really involved multiplication and addition to obtain the new probabilities at each location.